In [1]:
import cv2 as cv
import os
import numpy as np
from tqdm import tqdm
from math import sqrt
from math import atan
import math

In [2]:
img_dir = "./cobb_angle_img"
img_list = os.listdir(img_dir)
print(img_list)
output = "./cobb_angle_test"

['003.png', '007.png', '008.png', '012.png', '014.png', '019.png', '023.png', '024.png', '027.png', '029.png', '037.png', '043.png', '2.png', '31.png', '33.png']


In [3]:
yellow = (0, 255, 255)
red = (0, 0, 255)
green = (0, 255, 0)
blue = (255, 0, 0)
name_list = ['L5', 'L4', 'L3', 'L2', 'L1', 'T12', 'T11', 'T10', 'T9', 'T8', 'T7', 'T6', 'T5', 'T4', 'T3', 'T2', 'T1',
             'C7', 'C6', 'C5', 'C4', 'C3', 'C2', 'C1']
print(len(name_list))

24


In [4]:
def draw_upper_vertebra(vertebrae_information, color):
    for item in vertebrae_information:
        if item['location']:
            pt1 = (item['vertexes'][2][0], item['vertexes'][2][1])
            pt2 = (item['vertexes'][3][0], item['vertexes'][3][1])
            cv.line(im, pt1, pt2, color, 20)
        else:
            pt1 = (item['vertexes'][1][0], item['vertexes'][1][1])
            pt2 = (item['vertexes'][2][0], item['vertexes'][2][1])
            cv.line(im, pt1, pt2, color, 20)

In [5]:
def draw_turning_point(turning_location, vertebrae_information, color):
    for item in turning_location:
        location = vertebrae_information[item['previous']]
        if location['location']:
            pt1 = (location['vertexes'][2][0], location['vertexes'][2][1])
            pt2 = (location['vertexes'][3][0], location['vertexes'][3][1])
            cv.line(im, pt1, pt2, color, 20)
        else:
            pt1 = (location['vertexes'][1][0], location['vertexes'][1][1])
            pt2 = (location['vertexes'][2][0], location['vertexes'][2][1])
            cv.line(im, pt1, pt2, color, 20)

In [6]:
def draw_singal_vertebra(item, color, up):
    if up:
        if item['location']:
            pt1 = (item['vertexes'][2][0], item['vertexes'][2][1])
            pt2 = (item['vertexes'][3][0], item['vertexes'][3][1])
            cv.line(im, pt1, pt2, color, 20)
        else:
            pt1 = (item['vertexes'][1][0], item['vertexes'][1][1])
            pt2 = (item['vertexes'][2][0], item['vertexes'][2][1])
            cv.line(im, pt1, pt2, color, 20)
    else:
        if item['location']:
            pt1 = (item['vertexes'][0][0], item['vertexes'][0][1])
            pt2 = (item['vertexes'][1][0], item['vertexes'][1][1])
            cv.line(im, pt1, pt2, color, 20)
        else:
            pt1 = (item['vertexes'][0][0], item['vertexes'][0][1])
            pt2 = (item['vertexes'][3][0], item['vertexes'][3][1])
            cv.line(im, pt1, pt2, color, 20)

In [7]:
for item in tqdm(img_list):
    file = os.path.join(img_dir, item)
    output_file = os.path.join(output, item)

    im = cv.imread(file)
    imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
    ret, thresh = cv.threshold(imgray, 127, 255, 0)
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    # Vertebra rect 정보 목록, 작은 것에서 큰 것까지 y 값
    rect_list = []
    for index in range(len(contours)):
        contour = contours[index]
        rect = cv.minAreaRect(contour)  # 최소 사각형 (가운데 (x, y), (너비, 높이), 회전 각도) 가져 오기
        rect_list.append(rect)
    rect_list.sort(key=lambda item: item[0][1])  # 작은 것에서 큰 것까지 y 값으로 정렬

    mark_rect = rect_list.copy()
    mark_rect.reverse()

    # 척추 종점 정보, 중심점, 상부 척추 경사, 하부 척추 경사
    vertebrae_information = []
    turning_location = []
    for index in range(len(mark_rect)):
        center = (int(mark_rect[index][0][0] - 50), int(mark_rect[index][0][1]))
        im = cv.putText(im, str(name_list[index]), center, cv.FONT_HERSHEY_SIMPLEX, 2, blue, 3)

    for index in range(len(rect_list)):
        center = (int(rect_list[index][0][0] - 50), int(rect_list[index][0][1]))

        # 척추 정점 정보 저장
        box = cv.boxPoints(rect_list[index])
        box = np.int0(box)
        # 점 01의 위치가 수평인지 수직인지 확인
        point0 = (box[0][0], box[0][1])
        point1 = (box[1][0], box[1][1])
        point2 = (box[2][0], box[2][1])
        distance_01 = sqrt(pow(point0[0] - point1[0], 2) + pow(point0[1] - point1[1], 2))
        distance_12 = sqrt(pow(point1[0] - point2[0], 2) + pow(point1[1] - point2[1], 2))

        # 2 3
        # 1 0
        if distance_01 > distance_12:
            up_slope = (box[3][1] - box[2][1]) / (box[3][0] - box[2][0])
            down_slope = (box[0][1] - box[1][1]) / (box[0][0] - box[1][0])
            vertex = {'index': index, 'location': True, 'vertexes': box, 'center': center, 'up_slope': up_slope,
                      'down_slope': down_slope}
            vertebrae_information.append(vertex)
        # 1 2
        # 0 3
        else:
            up_slope = (box[2][1] - box[1][1]) / (box[2][0] - box[1][0])
            down_slope = (box[3][1] - box[0][1]) / (box[3][0] - box[0][0])
            vertex = {'index': index, 'location': False, 'vertexes': box, 'center': center, 'up_slope': up_slope,
                      'down_slope': down_slope}
            vertebrae_information.append(vertex)
        im = cv.drawContours(im, [box], 0, green, 1)

    # 휘어진 곳 찾기
    flag_index = [-1, -1]
    for index in range(len(rect_list)):
        if index + 1 < len(rect_list):
            if vertebrae_information[index]['location'] == False and vertebrae_information[index + 1][
                'location'] == True:
                if flag_index[1] != index:  # 제거 간격은 1
                    flag_index = [index, index + 1]
                    turning_index = {'previous': index, 'last': index + 1}
                    turning_location.append(turning_index)
            if vertebrae_information[index]['location'] == True and vertebrae_information[index + 1][
                'location'] == False:
                if flag_index[1] != index:
                    flag_index = [index, index + 1]
                    turning_index = {'previous': index, 'last': index + 1}
                    turning_location.append(turning_index)

    # 상부 및 하부 원뿔의 절대 값이 가장 큰 척추를 찾기
    # 그리고 최대 Cobb Angle을 계산
    slope_decline = vertebrae_information.copy()
    slope_decline.sort(key=lambda x: x['up_slope'], reverse=True)
    end_index = len(slope_decline) - 1
    max_cobb = []

    if slope_decline[0]['center'][1] > slope_decline[end_index]['center'][1]:
        draw_singal_vertebra(slope_decline[end_index], red, True)  # 상부 척추
        draw_singal_vertebra(slope_decline[0], red, False)  # 하부 척추

        k1 = slope_decline[end_index]['up_slope']
        k2 = slope_decline[0]['down_slope']
        angle = math.degrees(atan(abs((k2 - k1) / (1 + k1 * k2))))
        cobb = {'up_id': slope_decline[end_index]['index'], 'bottom_id': slope_decline[0]['index'], 'cobb_angle': angle}
        max_cobb.append(cobb)

        # 각도에 주석 달기
        y = int(slope_decline[end_index]['center'][1] + \
                (slope_decline[0]['center'][1] - slope_decline[end_index]['center'][1]) / 2)
        put_point = (slope_decline[0]['center'][0] - 500, y)
        im = cv.putText(im, str(round(angle, 2)), put_point, cv.FONT_HERSHEY_SIMPLEX, 4,
                        red, 4)
    else:
        draw_singal_vertebra(slope_decline[end_index], red, False)  # 하단 척추
        draw_singal_vertebra(slope_decline[0], red, True)  # 상단 척추

        k1 = slope_decline[0]['up_slope']
        k2 = slope_decline[end_index]['down_slope']
        angle = math.degrees(atan(abs((k2 - k1) / (1 + k1 * k2))))
        cobb = {'up_id': slope_decline[0]['index'], 'bottom_id': slope_decline[end_index]['index'], 'cobb_angle': angle}
        max_cobb.append(cobb)

        y = int(slope_decline[0]['center'][1] + \
                (slope_decline[end_index]['center'][1] - slope_decline[0]['center'][1]) / 2)
        put_point = (slope_decline[1]['center'][0] - 500, y)
        im = cv.putText(im, str(round(angle, 2)), put_point, cv.FONT_HERSHEY_SIMPLEX, 4,
                        red, 4)

    # 최대 Cobb Angle을 기준으로 나머지 각도를 계산
    for item in max_cobb:
        up_id = int(item['up_id'])
        bt_id = int(item['bottom_id'])
        print(up_id, bt_id)
        ver_top = vertebrae_information[0]
        ver_end = vertebrae_information[len(vertebrae_information) - 1]

        if item['up_id'] == 0 or item['bottom_id'] == (len(vertebrae_information) - 1):
            if item['up_id'] == 0:
                if item['bottom_id'] == (len(max_cobb) - 1):
                    print("전체가 굴곡")  # type0
                else:
                    print("나머지 하단 부분에서 두 개의 측만증")  # type1
                    # 아래쪽 절반 순서 변경
                    rest_series = vertebrae_information.copy()[bt_id - 1:]
                    rest_series.sort(key=lambda x: x['up_slope'], reverse=True)
                    bottom_id = len(rest_series) - 1

                    new_vertebrae = [rest_series[0], rest_series[bottom_id], ver_end]
                    new_vertebrae.sort(key=lambda x: x['center'][1])
                    # cobb의 하반부를 계산
                    k1 = new_vertebrae[0]['up_slope']
                    k2 = new_vertebrae[1]['down_slope']
                    k3 = new_vertebrae[1]['up_slope']
                    k4 = new_vertebrae[2]['down_slope']
                    cobb1 = math.degrees(atan(abs((k2 - k1) / (1 + k1 * k2))))
                    cobb2 = math.degrees(atan(abs((k4 - k3) / (1 + k4 * k3))))

                    texPoint1 = (new_vertebrae[0]['center'][0] - 500, new_vertebrae[0]['center'][1])
                    im = cv.putText(im, str(round(cobb1, 2)), texPoint1, cv.FONT_HERSHEY_SIMPLEX, 4,
                                    yellow, 4)
                    texPoint2 = (new_vertebrae[2]['center'][0] - 500, new_vertebrae[2]['center'][1])
                    im = cv.putText(im, str(round(cobb2, 2)), texPoint2, cv.FONT_HERSHEY_SIMPLEX, 4,
                                    green, 4)

                    draw_singal_vertebra(new_vertebrae[0], yellow, True)
                    draw_singal_vertebra(new_vertebrae[1], yellow, False)
                    draw_singal_vertebra(new_vertebrae[1], green, True)
                    draw_singal_vertebra(new_vertebrae[2], green, False)


            else:
                print("윗부분에서 두 개의 측만증")  # type2
                # 위쪽 절반 순서 변경
                rest_series = vertebrae_information.copy()[:up_id + 1]
                rest_series.sort(key=lambda x: x['up_slope'], reverse=True)
                bottom_id = len(rest_series) - 1

                new_vertebrae = [rest_series[0], rest_series[bottom_id], ver_top]
                new_vertebrae.sort(key=lambda x: x['center'][1])

                # cobb의 전반부를 계산
                k1 = new_vertebrae[0]['up_slope']
                k2 = new_vertebrae[1]['down_slope']
                k3 = new_vertebrae[1]['up_slope']
                k4 = new_vertebrae[2]['down_slope']
                cobb1 = math.degrees(atan(abs((k2 - k1) / (1 + k1 * k2))))
                cobb2 = math.degrees(atan(abs((k4 - k3) / (1 + k4 * k3))))

                texPoint1 = (new_vertebrae[0]['center'][0] - 500, new_vertebrae[0]['center'][1])
                im = cv.putText(im, str(round(cobb1, 2)), texPoint1, cv.FONT_HERSHEY_SIMPLEX, 4,
                                yellow, 4)
                texPoint2 = (new_vertebrae[2]['center'][0] - 500, new_vertebrae[2]['center'][1])
                im = cv.putText(im, str(round(cobb2, 2)), texPoint2, cv.FONT_HERSHEY_SIMPLEX, 4,
                                green, 4)

                draw_singal_vertebra(new_vertebrae[0], yellow, True)
                draw_singal_vertebra(new_vertebrae[1], yellow, False)
                draw_singal_vertebra(new_vertebrae[1], green, True)
                draw_singal_vertebra(new_vertebrae[2], green, False)

        else:
            print("중간에서 상단 및 하단 부위에서 측만증")  # type3
            # 가로 채기의 양쪽에서 별도로 계산
            top_series = vertebrae_information.copy()[:up_id + 1]
            bottom_series = vertebrae_information.copy()[bt_id - 1:]
            tp_end = len(top_series) - 1
            bt_end = len(bottom_series) - 1

            top_series.sort(key=lambda x: x['up_slope'], reverse=True)
            bottom_series.sort(key=lambda x: x['up_slope'], reverse=True)

            new_vertebrae1 = [top_series[0], top_series[tp_end]]
            new_vertebrae1.sort(key=lambda x: x['center'][1])
            new_vertebrae2 = [bottom_series[0], bottom_series[bt_end]]
            new_vertebrae2.sort(key=lambda x: x['center'][1])

            # cobb의 하반부를 계산
            k1 = new_vertebrae1[0]['up_slope']
            k2 = new_vertebrae1[1]['down_slope']
            k3 = new_vertebrae2[0]['up_slope']
            k4 = new_vertebrae2[1]['down_slope']
            cobb1 = math.degrees(atan(abs((k2 - k1) / (1 + k1 * k2))))
            cobb2 = math.degrees(atan(abs((k4 - k3) / (1 + k4 * k3))))

            texPoint1 = (new_vertebrae1[0]['center'][0] - 500, new_vertebrae1[0]['center'][1])
            im = cv.putText(im, str(round(cobb1, 2)), texPoint1, cv.FONT_HERSHEY_SIMPLEX, 4,
                            yellow, 4)
            texPoint2 = (new_vertebrae2[1]['center'][0] - 500, new_vertebrae2[1]['center'][1])
            im = cv.putText(im, str(round(cobb2, 2)), texPoint2, cv.FONT_HERSHEY_SIMPLEX, 4,
                            green, 4)

            draw_singal_vertebra(new_vertebrae1[0], yellow, True)
            draw_singal_vertebra(new_vertebrae1[1], yellow, False)
            draw_singal_vertebra(new_vertebrae2[0], green, True)
            draw_singal_vertebra(new_vertebrae2[1], green, False)

    cv.imwrite(output_file, im)
print("작업 완료")

  7%|█████▌                                                                             | 1/15 [00:00<00:03,  3.58it/s]

10 14
중간에서 상단 및 하단 부위에서 측만증


 13%|███████████                                                                        | 2/15 [00:00<00:03,  3.95it/s]

11 16
중간에서 상단 및 하단 부위에서 측만증


 20%|████████████████▌                                                                  | 3/15 [00:00<00:02,  4.05it/s]

...

10 23
윗부분에서 두 개의 측만증


 93%|████████████████████████████████████████████████████████████████████████████▌     | 14/15 [00:03<00:00,  3.04it/s]

0 22
나머지 하단 부분에서 두 개의 측만증


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:03<00:00,  3.88it/s]

15 21
윗부분에서 두 개의 측만증
작업 완료
